In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
(train_input, train_target),(test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [3]:
# train과 valid 분리
from sklearn.model_selection import train_test_split

train_scaled = train_input /255.0
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size = 0.2, random_state= 42
)

<img src = '../Data/심층신경망.png' width=600 heigth=250>

> 은닉층은 출력층보다 많아야된다. 몇개로 해야되나?(이건 hyper parameter이다.)

---
### Layer를 추가하는 방법 - 1

In [4]:
# Layer 만들기
    # Dense : 밀집층(덴스) 
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) # 은닉층생성
dense2 = keras.layers.Dense(10, activation='softmax') # 출력층

In [5]:
# 전체모델에 층을 추가
model = keras.Sequential([dense1, dense2])

2022-07-20 14:46:58.296732: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Summary
    # 가중치 갯수 Param : 784*100 + 100(절편)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### Layer를 추가하는 방법 - 2

In [7]:
model = keras.Sequential(
    [
        keras.layers.Dense(100, activation ='sigmoid', input_shape=(784,), name ='hidden'),
        keras.layers.Dense(10, activation = 'softmax', name ='output')
    ], name = '패션 Mnist 모델'
)

In [8]:
model.summary()

Model: "패션 Mnist 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### Layer를 추가하는 방법 - 3

In [9]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### 모델 훈련

In [12]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5631 - accuracy: 0.8083
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4106 - accuracy: 0.8516
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3761 - accuracy: 0.8651
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3535 - accuracy: 0.8727
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3366 - accuracy: 0.8775


sigmoid 함수 -> relu 함수 
<img src = '../Data/relu.png' width=400 heigth=150>

- Sigmoid 함수는 숫자가 무한대로 커지게 되면 데이터의 차이가 거의 없다는 단점이있다.
- 이를 해결한 것이 Relu 함수인데 인공신경망에 기여한 바가 크며 음수는 모든 0으로 양수는 항상 최대값을 출력하는 함수

In [13]:
# 입력층을 단순화 하기
model = keras.Sequential()
# 입력층을 2차원 원래대로 사용한다.
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### Optimizer(최적화 알고리즘)
- 훈련시에 Optimizer를 사용하여 최적화 알고리즘을 구성한다.
- 보통 사용하는 것이 확률적 경사하강법 optimizer이다.

> 옵티마이저는 정방향에서 역방향이 될때 사용하는 것이다.<br>

<img src = '../Data/Adam.png' width=600 heigth=250>

---
##### SGD

In [17]:
# 적용방법 (확률적 경사 하강법) - 1
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [18]:
# 적용방법 (확률적 경사 하강법) - 2
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')


In [19]:
# 적용방법 (확률적 경사 하강법) + 학습률 변경 - 3 : SGD의 learning_rate의 기본값은 0.01

sgd = keras.optimizers.SGD(learning_rate=0.1)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

---
##### Momentum

In [22]:
# 적용방법 (Momentum)
    # momentum은 sgd에서 발달되어 끝까지 0인 값까지 갔다가 다시 return해서 맞춘다.
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

---
##### Adagrad

In [23]:
# 적용방법 (Adagrad)
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

---
##### RMSprop

In [24]:
# 적용방법 (RMSprop)
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')